In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('assam_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/dhemaji-to-jorhat: Message: 
Stacktrace:
0   chromedriver                        0x0000000104bc39a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000104bbbe10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x00000001047c0be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000104804f10 cxxbridge1$string$len + 368852
4   chromedriver                        0x000000010483e4a4 cxxbridge1$string$len + 603752
5   chromedriver                        0x00000001047f9a08 cxxbridge1$string$len + 322508
6   chromedriver                        0x00000001047fa66c cxxbridge1$string$len + 325680
7   chromedriver                        0x0000000104b8a068 cxxbridge1$str$ptr + 1651256
8   chromedriver                        0x0000000104b8e99c cxxbridge1$str$ptr + 1669996
9   chromedriver                        0x0000000104b6f1fc cxxbridge1$str$ptr 

NameError: name 'driver' is not defined

In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 156885,Bharat Benz A/C Seater (2+2),08:00,04h 30m,12:30,4.5,INR 298,29 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 153223,Bharat Benz A/C Seater (2+2),09:15,04h 45m,14:00,3.7,INR 298,19 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 170878,Bharat Benz A/C Seater (2+2),10:35,04h 40m,15:15,4.6,INR 298,19 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,04h 15m,20:00,2.9,INR 298,17 Seats available
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
328,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Maa Laxmi,NON A/C Seater Push Back (2+1),17:30,04h 45m,22:15,3.0,INR 500,36 Seats available
329,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Baikuntha Transport Service,Bharat Benz A/C Sleeper (2+1),19:30,05h 30m,01:00,3.9,500,50 Seats available
330,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Purple Wings Coaches,A/C Seater / Sleeper (2+1),19:55,05h 20m,01:15,3.6,569,43 Seats available
331,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Shree Ram Travels (Under ASTC),Bharat Benz A/C Seater (2+1),20:00,05h 00m,01:00,4.8,500,37 Seats available
